In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files
uploaded = files.upload()
zip_path   = "/content/Medical Image Classification.zip"
extract_to = "/content/medical_cls"

import os, zipfile

os.makedirs(extract_to, exist_ok=True)

with zipfile.ZipFile(zip_path, 'r') as zf:
    zf.extractall(extract_to)

!ls -R "{extract_to}"


8 classes model

In [ ]:
import os, glob, random
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torchvision.models as models
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("DEVICE:", DEVICE)

IMG_SIZE   = 128
BATCH_SIZE = 32

train_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.RandomHorizontalFlip(),
    T.RandomRotation(10),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5],
                std=[0.5, 0.5, 0.5])
])

test_transform = T.Compose([
    T.Resize((IMG_SIZE, IMG_SIZE)),
    T.ToTensor(),
    T.Normalize(mean=[0.5, 0.5, 0.5],
                std=[0.5, 0.5, 0.5])
])

root = "/content/medical_cls"   # zip'i buraya açmıştık
organs = ["brain", "chest", "kidney", "lung"]
subtypes = ["real", "fake"]

class_names = [
    "brain_real", "brain_fake",
    "chest_real", "chest_fake",
    "kidney_real", "kidney_fake",
    "lung_real", "lung_fake"
]
print("Sınıflar:", class_names)
num_classes = len(class_names)


DEVICE: cuda
Sınıflar: ['brain_real', 'brain_fake', 'chest_real', 'chest_fake', 'kidney_real', 'kidney_fake', 'lung_real', 'lung_fake']


In [ ]:
all_samples_8 = []   # (img_path, class_idx)

for organ_idx, organ in enumerate(organs):
    for subtype_idx, subtype in enumerate(subtypes):
        cls_name = f"{organ}_{subtype}"
        cls_idx  = class_names.index(cls_name)

        img_dir = os.path.join(root, organ, subtype)
        paths = []
        for ext in ("*.jpg", "*.jpeg", "*.png"):
            paths.extend(glob.glob(os.path.join(img_dir, ext)))
        paths = sorted(paths)

        print(f"{cls_name} -> {len(paths)} images")

        for p in paths:
            all_samples_8.append((p, cls_idx))

print("Total sample count (8 sınıf):", len(all_samples_8))
print("Example:", all_samples_8[0] if all_samples_8 else "Not")


In [ ]:
paths = [p for (p, c) in all_samples_8]
labels = [c for (p, c) in all_samples_8]

# önce train + temp
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    paths, labels, test_size=0.3, random_state=42, stratify=labels
)

# temp'i val + test olarak böl
val_paths, test_paths, val_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

print("Train:", len(train_paths))
print("Val  :", len(val_paths))
print("Test :", len(test_paths))


In [ ]:
class Medical8ClassDataset(Dataset):
    def __init__(self, paths, labels, transform):
        self.paths = paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, idx):
        img_path = self.paths[idx]
        label    = self.labels[idx]
        img = Image.open(img_path).convert("RGB")
        img = self.transform(img)
        return img, label

train_set_8 = Medical8ClassDataset(train_paths, train_labels, train_transform)
val_set_8   = Medical8ClassDataset(val_paths,   val_labels,   test_transform)
test_set_8  = Medical8ClassDataset(test_paths,  test_labels,  test_transform)

train_loader_8 = DataLoader(train_set_8, batch_size=BATCH_SIZE, shuffle=True,  num_workers=2)
val_loader_8   = DataLoader(val_set_8,   batch_size=BATCH_SIZE, shuffle=False, num_workers=2)
test_loader_8  = DataLoader(test_set_8,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

print("Train batches:", len(train_loader_8))
print("Val   batches:", len(val_loader_8))
print("Test  batches:", len(test_loader_8))


In [ ]:
resnet8 = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)

# Son katmanı 8 sınıfa uyarlayalım
in_features = resnet8.fc.in_features
resnet8.fc = nn.Linear(in_features, num_classes)

resnet8 = resnet8.to(DEVICE)

criterion_8 = nn.CrossEntropyLoss()
optimizer_8 = torch.optim.Adam(resnet8.parameters(), lr=1e-4)


In [ ]:
def run_epoch_8(model, loader, optimizer=None):
    if optimizer is None:
        model.eval()
    else:
        model.train()

    total_loss = 0.0
    total_correct = 0
    total_samples = 0

    for imgs, labels in loader:
        imgs   = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = model(imgs)
        loss = criterion_8(outputs, labels)

        if optimizer is not None:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * imgs.size(0)
        preds = outputs.argmax(dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += imgs.size(0)

    avg_loss = total_loss / total_samples
    avg_acc  = total_correct / total_samples
    return avg_loss, avg_acc

EPOCHS_8 = 15
best_val_acc_8 = 0.0
best_path_8 = "/content/resnet_8class_best.pth"

for epoch in range(1, EPOCHS_8+1):
    train_loss, train_acc = run_epoch_8(resnet8, train_loader_8, optimizer=optimizer_8)
    val_loss,   val_acc   = run_epoch_8(resnet8, val_loader_8,   optimizer=None)

    print(f"Epoch {epoch}/{EPOCHS_8} | "
          f"TrainLoss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"ValLoss: {val_loss:.4f} Acc: {val_acc:.4f}")

    if val_acc > best_val_acc_8:
        best_val_acc_8 = val_acc
        torch.save(resnet8.state_dict(), best_path_8)
        print("  -> Yeni en iyi model kaydedildi:", best_path_8)

print("En iyi val acc (8 sınıf):", best_val_acc_8)


In [ ]:
# En iyi modeli yükle
resnet8.load_state_dict(torch.load(best_path_8, map_location=DEVICE))

test_loss_8, test_acc_8 = run_epoch_8(resnet8, test_loader_8, optimizer=None)
print(f"8-CLASS TEST | Loss: {test_loss_8:.4f}  Acc: {test_acc_8:.4f}")


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

resnet8.eval()
all_labels = []
all_preds  = []

with torch.no_grad():
    for imgs, labels in test_loader_8:
        imgs   = imgs.to(DEVICE)
        labels = labels.to(DEVICE)

        outputs = resnet8(imgs)
        preds   = outputs.argmax(dim=1)

        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

all_labels = np.array(all_labels)
all_preds  = np.array(all_preds)

print("\n🔍 CLASSIFICATION REPORT (8 sınıf, test seti)\n")
print(classification_report(all_labels, all_preds, target_names=class_names, digits=4))

cm = confusion_matrix(all_labels, all_preds)

plt.figure(figsize=(8,7))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel("Predicted")
plt.ylabel("True")
plt.title("Confusion Matrix - 8-Class Test")
plt.show()
